# Text Generation

In [1]:
import pandas as pd
import torch
import torch.nn.functional as F
from transformers import AutoTokenizer, AutoModelForCausalLM

In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model_name = 'gpt2'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name).to(device)

Downloading:   0%|          | 0.00/665 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/0.99M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.29M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/523M [00:00<?, ?B/s]

## Greedy Search Decoding

In [3]:
input_txt = 'Transformers are the '
input_ids = tokenizer(input_txt, return_tensors='pt')['input_ids'].to(device)
iterations = []
n_steps = 8
choices_per_step = 5

with torch.no_grad():
    for _ in range(n_steps):
        iteration = {'input': tokenizer.decode(input_ids[0])}
        output = model(input_ids=input_ids)

        next_token_logits = output.logits[0, -1, :]
        next_token_probas = torch.softmax(next_token_logits, dim=-1)
        sorted_ids = torch.argsort(next_token_probas, dim=-1, descending=True)

        for choice_idx in range(choices_per_step):
            token_id = sorted_ids[choice_idx]
            token_prob = next_token_probas[token_id].cpu().numpy()
            token_choice = (
                f'{tokenizer.decode(token_id)} ({100 * token_prob: .2f}%'
            )
            iteration[f'Choice {choice_idx+1}'] = token_choice

        input_ids = torch.cat([input_ids, sorted_ids[None, 0, None]], dim=-1)
        iterations.append(iteration)

df = pd.DataFrame(iterations)
df

,input,Choice 1,Choice 2,Choice 3,Choice 4,Choice 5
0,Transformers are the,vern ( 14.87%,( 11.98%,ids ( 7.52%,ices ( 4.99%,urs ( 4.02%
1,Transformers are the vern,acular ( 94.56%,al ( 3.49%,ier ( 1.01%,iers ( 0.16%,us ( 0.08%
2,Transformers are the vernacular,of ( 28.65%,for ( 17.40%,term ( 5.95%,name ( 5.18%,words ( 2.47%
3,Transformers are the vernacular of,the ( 22.59%,all ( 1.54%,a ( 1.45%,our ( 0.75%,this ( 0.69%
4,Transformers are the vernacular of the,game ( 1.31%,time ( 1.14%,world ( 1.08%,modern ( 0.92%,ancient ( 0.71%
5,Transformers are the vernacular of the game,. ( 25.34%,", ( 22.62%",'s ( 8.07%,and ( 5.68%,world ( 4.95%
6,Transformers are the vernacular of the game.,\n ( 15.49%,They ( 11.45%,The ( 8.13%,In ( 2.92%,It ( 2.64%
7,Transformers are the vernacular of the game.\n,\n ( 99.65%,The ( 0.04%,A ( 0.02%,I ( 0.01%,In ( 0.01%


In [4]:
input_ids = tokenizer(input_txt, return_tensors='pt')['input_ids'].to(device)
output = model.generate(input_ids, max_new_tokens=n_steps, do_sample=False)
print(tokenizer.decode(output[0]))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Transformers are the vernacular of the game.




## Beam Search Decoding

In [5]:
max_length = 128
input_txt = """In a shocking finding, scientist discovered \
a herd of unicorns living in a remote, previously unexplored \
valley, in the Andes Mountains. Even more suprising to the \
researchers was the fact that the unicorns spoke perfect English.\n\n
"""
input_ids = tokenizer(input_txt, return_tensors='pt')['input_ids'].to(device)
output = model.generate(input_ids, max_length=max_length, do_sample=False)
print(tokenizer.decode(output[0]))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In a shocking finding, scientist discovered a herd of unicorns living in a remote, previously unexplored valley, in the Andes Mountains. Even more suprising to the researchers was the fact that the unicorns spoke perfect English.


"The unicorns were very intelligent, and they were very intelligent," said Dr. David S. Siegel, a professor of anthropology at the University of California, Berkeley. "They were very intelligent, and they were very intelligent. They were very intelligent, and they were very intelligent."


The researchers found that the unicorns were able to communicate with each other through their vocal cords, which


In [16]:
def log_probs_from_logits(logits, labels):
    logp = F.log_softmax(logits, dim=-1)
    logp_label = torch.gather(logp, dim=2, index=labels.unsqueeze(2)).squeeze(-1)
    return logp_label


def sequence_logprob(model, labels, input_len=0):
    with torch.no_grad():
        output = model(labels)
        log_probs = log_probs_from_logits(output.logits[:, :-1, :], labels[:, 1:])
        seq_logprob = torch.sum(log_probs[:, input_len:])
    return seq_logprob.cpu().numpy()

In [17]:
# greedy search
logp = sequence_logprob(model, output, input_len=len(input_ids[0]))
print(tokenizer.decode(output[0]))
print(f'log-prob: {logp: .2f}')

In a shocking finding, scientist discovered a herd of unicorns living in a remote, previously unexplored valley, in the Andes Mountains. Even more suprising to the researchers was the fact that the unicorns spoke perfect English.


Advertisement

Now, researchers are asking whether the unicorns might even have been part of a mysterious and extremely elaborate civilization of prehistoric beings. The results suggest that they might even be extinct.


The researchers looked at the genome of two male and two female unicorns in the Andes Mountains, which is the only part of the world without unicorns. The two animals lived in herds, but
log-prob: -171.05


In [18]:
# beam search
output = model.generate(input_ids, max_length=max_length, num_beams=5, do_sample=False)
logp = sequence_logprob(model, output, input_len=len(input_ids[0]))
print(tokenizer.decode(output[0]))
print(f'log-prob: {logp: .2f}')

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In a shocking finding, scientist discovered a herd of unicorns living in a remote, previously unexplored valley, in the Andes Mountains. Even more suprising to the researchers was the fact that the unicorns spoke perfect English.


According to the researchers, the unicorns were able to communicate with each other in a way that was similar to that of humans. The unicorns were able to communicate with each other in a way that was similar to that of humans. The unicorns were able to communicate with each other in a way that was similar to that of humans. The unicorns were able to communicate with each other in a
log-prob: -47.81


In [19]:
# beam search, no repeat ngrams
output = model.generate(
    input_ids, max_length=max_length, num_beams=5, do_sample=False, no_repeat_ngram_size=2
)
logp = sequence_logprob(model, output, input_len=len(input_ids[0]))
print(tokenizer.decode(output[0]))
print(f'log-prob: {logp: .2f}')

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In a shocking finding, scientist discovered a herd of unicorns living in a remote, previously unexplored valley, in the Andes Mountains. Even more suprising to the researchers was the fact that the unicorns spoke perfect English.


The researchers, from the University of California, San Diego, and the National Science Foundation (NSF) in Boulder, Colorado, were able to translate the words of the unicorn into English, which they then translated into Spanish.

"This is the first time that we have translated a language into an English language," said study co-author and NSF professor of linguistics and evolutionary biology Dr
log-prob: -102.25


In [20]:
# top k samp,ing
output = model.generate(
    input_ids, max_length=max_length, do_sample=True, top_k=50
)
logp = sequence_logprob(model, output, input_len=len(input_ids[0]))
print(tokenizer.decode(output[0]))
print(f'log-prob: {logp: .2f}')


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In a shocking finding, scientist discovered a herd of unicorns living in a remote, previously unexplored valley, in the Andes Mountains. Even more suprising to the researchers was the fact that the unicorns spoke perfect English.


"They were actually able to tell if their language contained real language," says Toni M. Janssen, a University of Pennsylvania linguist who spoke with the New York Times. "They started using it as they approached the area. The only missing word is "fantastic" or "kind."

The unicorns are believed to be from the eastern and northern Rockies of Mexico,
log-prob: -203.54


In [21]:
# top k and nucleus sampling
output = model.generate(
    input_ids, max_length=max_length, do_sample=True, top_k=50, top_p=0.9
)
logp = sequence_logprob(model, output, input_len=len(input_ids[0]))
print(tokenizer.decode(output[0]))
print(f'log-prob: {logp: .2f}')

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In a shocking finding, scientist discovered a herd of unicorns living in a remote, previously unexplored valley, in the Andes Mountains. Even more suprising to the researchers was the fact that the unicorns spoke perfect English.


The results show that the unicorns actually prefer the familiar English language, but do not share a common ancestor with human beings. Moreover, the unicorns are very fast and they can reach heights up to 600 meters.


In fact, the researchers observed that this amazing animal had more in common with humans than they expected. Although they do not believe humans are directly related, the fact is that these
log-prob: -187.75
